#### 7.	consider a file with a column named Education, which has array of elements as shown below. Convert element in the array to record using spark dataframe.
inputFile:
Name|Age|Education
Naresh|24|B.Tech,Inter,SSC
Hari|32
Nani|31|M.Tech,B.Tech,Inter,SSC
Kumar|27|P.H.D,M.B.A,Degree,Inter,SSC

In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('education').getOrCreate()
df = spark.read.option("delimiter","|").csv("education.csv",header=True)
df.show(truncate=False)

+------+---+----------------------------+
|Name  |Age|Education,                  |
+------+---+----------------------------+
|Naresh|24 |B.Tech,Inter,SSC,           |
|Hari  |32 |null                        |
|Nani  |31 |M.Tech,B.Tech,Inter,SSC,    |
|Kumar |27 |P.H.D,M.B.A,Degree,Inter,SSC|
+------+---+----------------------------+



## explode: from multiple column values into single values: Null values not covered

In [25]:
from pyspark.sql.functions import explode, split

df.withColumn("Qualification",explode(split("Education,",","))).drop("Education,").show()

+------+---+-------------+
|  Name|Age|Qualification|
+------+---+-------------+
|Naresh| 24|       B.Tech|
|Naresh| 24|        Inter|
|Naresh| 24|          SSC|
|Naresh| 24|             |
|  Nani| 31|       M.Tech|
|  Nani| 31|       B.Tech|
|  Nani| 31|        Inter|
|  Nani| 31|          SSC|
|  Nani| 31|             |
| Kumar| 27|        P.H.D|
| Kumar| 27|        M.B.A|
| Kumar| 27|       Degree|
| Kumar| 27|        Inter|
| Kumar| 27|          SSC|
+------+---+-------------+



## explode_outer with Null values covered 

In [26]:
from pyspark.sql.functions import explode_outer, split

df.withColumn("Qualification",explode_outer(split("Education,",","))).drop("Education,").show()

+------+---+-------------+
|  Name|Age|Qualification|
+------+---+-------------+
|Naresh| 24|       B.Tech|
|Naresh| 24|        Inter|
|Naresh| 24|          SSC|
|Naresh| 24|             |
|  Hari| 32|         null|
|  Nani| 31|       M.Tech|
|  Nani| 31|       B.Tech|
|  Nani| 31|        Inter|
|  Nani| 31|          SSC|
|  Nani| 31|             |
| Kumar| 27|        P.H.D|
| Kumar| 27|        M.B.A|
| Kumar| 27|       Degree|
| Kumar| 27|        Inter|
| Kumar| 27|          SSC|
+------+---+-------------+



## position with values posexplode_outer

In [33]:
from pyspark.sql.functions import posexplode_outer, split

df.select("*", posexplode_outer(split("Education,",","))).withColumnRenamed("col","Qualification").withColumnRenamed("pos","Index").drop("Education,").show()

+------+---+-----+-------------+
|  Name|Age|Index|Qualification|
+------+---+-----+-------------+
|Naresh| 24|    0|       B.Tech|
|Naresh| 24|    1|        Inter|
|Naresh| 24|    2|          SSC|
|Naresh| 24|    3|             |
|  Hari| 32| null|         null|
|  Nani| 31|    0|       M.Tech|
|  Nani| 31|    1|       B.Tech|
|  Nani| 31|    2|        Inter|
|  Nani| 31|    3|          SSC|
|  Nani| 31|    4|             |
| Kumar| 27|    0|        P.H.D|
| Kumar| 27|    1|        M.B.A|
| Kumar| 27|    2|       Degree|
| Kumar| 27|    3|        Inter|
| Kumar| 27|    4|          SSC|
+------+---+-----+-------------+

